# Code 5
- Combine all Strategy
  - Missing Value Imputation
  - Outlier Startegy
  - Tranformation Strategy

## 1/ Import Libraries

In [1]:
#CodeSection1
# from google.colab import drive
# drive.mount('/mntDrive') 

In [2]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [3]:
#CodeSection3
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

## 3/ Combine both Datasets (train, test)

In [4]:
#CodeSection4
all_data = [train, test]

### Check Combined Data 
- Missing Value
- Data Types

In [5]:
#CodeSection5
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

id                           0
Gender                       0
Age                          0
Driving_License              0
Region_Code                  0
Previously_Insured           0
Vehicle_Age                  0
Vehicle_Damage               0
Annual_Premium               0
Policy_Sales_Channel         0
Vintage                      0
Response                127037
dtype: int64

In [6]:
#CodeSection6
# Check Data types for each variable
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


## 4/ Numerical Processing

### 4.1/ Impute Numerical Variables
-NA

### 4.2/ Outlier Strategy

In [7]:
#CodeSection11
numerical_features = ['Age','Driving_License','Region_Code','Annual_Premium','Policy_Sales_Channel','Vintage']
# Write a loop to detect and impute Outliers
for num_var in numerical_features:
    for dataset in all_data:

        Q1 = dataset[num_var].quantile(0.25)
        Q3 = dataset[num_var].quantile(0.75)

        IQR = Q3-Q1

        Lower_Whisker = Q1 - 1.5*IQR
        Upper_Whisker = Q3 + 1.5*IQR

        dataset[num_var] = dataset[num_var].apply(lambda x : Upper_Whisker if x > Upper_Whisker else x)

### 4.3/ Transformation Strategy

In [8]:
#CodeSection12
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

In [10]:
#CodeSection13

#Initiate StandardScaler
num_scale = StandardScaler()

# Fit StandardScaler
num_scale.fit(train[numerical_features])

StandardScaler()

In [11]:
#CodeSection14
# Scale Train and Test
for datasets in all_data:
    datasets[numerical_features]=num_scale.transform(datasets[numerical_features])

## 5/ Impute Categorical Variables

### Impute all Categorical Variables at once
-NA

## 6/ Now Re Check all Features

In [12]:
#CodeSection19
# Combine both to check Missing Value
pd.concat([train, test], axis = 0).isnull().sum()

id                           0
Gender                       0
Age                          0
Driving_License              0
Region_Code                  0
Previously_Insured           0
Vehicle_Age                  0
Vehicle_Damage               0
Annual_Premium               0
Policy_Sales_Channel         0
Vintage                      0
Response                127037
dtype: int64

## 8/ Prepare Data for Model Model

In [13]:
#CodeSection20
# Check First 5 Rows of Test Data
test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,-0.891113,0.046208,-1.163187,1,< 1 Year,No,0.364286,0.737321,-1.211258
1,381111,Male,0.075906,0.046208,0.121784,0,1-2 Year,Yes,0.233506,-1.937762,-0.518068
2,381112,Male,0.527181,0.046208,0.121784,0,1-2 Year,Yes,0.639803,0.220753,0.533668
3,381113,Male,-0.955581,0.046208,0.046198,1,< 1 Year,Yes,0.465731,0.737321,0.390249
4,381114,Male,-0.762177,0.046208,0.121784,1,< 1 Year,No,1.870517,0.737321,1.704919


In [14]:
#CodeSection21
# Get X and y Variables for your Dataset
X = train.drop(['id','Response'], axis =  1)
y = train['Response']

### Create Dummy Variables for  Categorical Variables

In [15]:
#CodeSection22
# Get Dummy Variables
X = pd.get_dummies(X, drop_first = True)  # Keep only n - 1 Variables - Use "drop_first = True"

## 9/ Build Model

In [16]:
#CodeSection23
#Import Logistic Regression Library
from sklearn.linear_model import LogisticRegression

#Initialize LR
clf = LogisticRegression(max_iter = 1000, class_weight='balanced')

In [17]:
#CodeSection24
# Fit Model
model = clf.fit(X, y)

### Check Accuracy of Model on Train Data

In [18]:
#CodeSection25
# Predict on Train Data
y_pred = model.predict(X)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [19]:
#CodeSection26
# Import metrics library
from sklearn.metrics import roc_auc_score

In [21]:
#CodeSection27
# get Actual "y" variables (use "y_true" as variable)
y_true = train['Response']

In [22]:
#CodeSection28
# Print Train Accuracy
print(f" Train Accuracy : {roc_auc_score(y_true, y_pred):0.1%}")

 Train Accuracy : 78.3%


## 10/ Predict and Submit

### Predict

In [23]:
#CodeSection29
# Get all the X Variables from the Test Dataset
X_test = test.drop(['id'], axis =  1)

In [24]:
#CodeSection30
#Covert to Dummies
X_test = pd.get_dummies(X_test, drop_first = True)

In [25]:
#CodeSection32
# Predict on X_test Data ("X_test_prep")
X_test_prep = model.predict(X_test)

### Create Submission File

In [27]:
#CodeSection33
submission = pd.DataFrame({
    'id' : test['id'],
    'Response' : X_test_prep
})

### Export Submission File

In [28]:
#CodeSection34
submission.to_csv('output/O5_Preproccesing_All.csv', index = False)

In [ ]:
# LB Accuracy :  0.7841382100082412